# Set up the Edgar Postgres DB

### Set up imports

In [18]:
import os
import psycopg2

### Set up database connection

Make sure the correct variables exist in your environment.

In [19]:
# Load credentials from environment. 
POSTGRES_ADDRESS = os.environ['POSTGRES_ADDRESS']
POSTGRES_PORT = os.environ['POSTGRES_PORT']
POSTGRES_USERNAME = os.environ['POSTGRES_USERNAME']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']
POSTGRES_DBNAME = os.environ['POSTGRES_DBNAME']

In [20]:
# Create connection and cursor    
conn = psycopg2.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
cur = conn.cursor()

### Create the profile, income, and balance sheet tables

(Only run this if it doesn't already exist)

In [4]:
# Create the Profile table
cur.execute("""CREATE TABLE profile_table
               (symbol varchar(5),
                price float,
                beta float,
                volAvg bigint,
                mktCap bigint,
                lastDiv float,
                changes float,
                companyName varchar(100),
                exchangeShortName varchar(10),
                industry varchar(100),
                website varchar(100),
                description text,
                ceo varchar(100),
                sector varchar(100),
                country varchar(20),
                fullTimeEmployees integer,
                phone varchar(20),
                address text,
                city varchar(50),
                state varchar(20),
                dcf float,
                ipoDate timestamp);""") 

# Commit the table creation transaction.
conn.commit()

In [5]:
# Create the Income Statement table
cur.execute("""CREATE TABLE income_table
                (symbol varchar(5),
                date timestamp,
                period varchar(5),
                revenue bigint, 
                costOfRevenue bigint, 
                grossProfit bigint, 
                grossProfitRatio float, 
                researchAndDevelopmentExpenses bigint, 
                generalAndAdministrativeExpenses bigint, 
                sellingAndMarketingExpenses bigint,
                otherExpenses bigint, 
                operatingExpenses bigint, 
                costAndExpenses bigint, 
                interestExpense bigint, 
                depreciationAndAmortization bigint, 
                ebitda bigint, 
                ebitdaratio float, 
                operatingIncome bigint, 
                operatingIncomeRatio float,
                totalOtherIncomeExpensesNet bigint, 
                incomeBeforeTax bigint, 
                incomeBeforeTaxRatio float, 
                incomeTaxExpense bigint, 
                netIncome bigint, 
                netIncomeRatio float, 
                eps float, 
                epsdiluted float, 
                weightedAverageShsOut bigint, 
                weightedAverageShsOutDil bigint);""") 
    
# Commit the table creation transaction.
conn.commit()

In [7]:
# Create the Balance Sheet Statement table
cur.execute("""CREATE TABLE balance
                (symbol varchar(5),
                date timestamp,
                period varchar(5),
                cashAndCashEquivalents bigint,
                shortTermInvestments bigint,
                cashAndShortTermInvestments bigint,
                netReceivables bigint,
                inventory bigint,
                totalCurrentAssets bigint,
                propertyPlantEquipmentNet bigint,
                goodwill bigint,
                intangibleAssets bigint,
                longTermInvestments bigint,
                taxAssets bigint,
                totalNonCurrentAssets bigint,
                otherAssets bigint,
                totalAssets bigint,
                accountPayables bigint,
                shortTermDebt bigint,
                taxPayables bigint,
                deferredRevenue bigint,
                totalCurrentLiabilities bigint,
                longTermDebt bigint,
                deferredRevenueNonCurrent bigint,
                totalLiabilities bigint,
                commonStock bigint,
                retainedEarnings bigint,
                totalStockholdersEquity bigint,
                totalLiabilitiesAndStockholdersEquity bigint,
                totalInvestments bigint,
                totalDebt bigint,
                netDebt bigint);""") 

# Commit the table creation transaction.
conn.commit()

### Display contents of the income table

Feel free to run this interactively to debug table population.

In [33]:
conn.commit()
query = """
Select symbol, totaldebt as result from balance WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY result desc limit 10;
"""
cur.execute(query)
cur.fetchall()

[('JPM', 326893000000),
 ('C', 301200000000),
 ('BAC', 282255000000),
 ('MS', 217079000000),
 ('F', 161684000000),
 ('T', 155209000000),
 ('VZ', 129062000000),
 ('BRK-B', 116895000000),
 ('AAPL', 112436000000),
 ('GM', 109894000000)]

In [23]:
conn.commit()
query = """
Select symbol, revenue as result
From income_table WHERE EXTRACT(YEAR FROM date) = 2020 and symbol = 'AXP';
"""
cur.execute(query)
cur.fetchall()

[('AXP', 26816000000)]

In [ ]:
conn.commit()
query = """
Select symbol, cast(operatingExpenses as float) / NULLIF(revenue, 0) as result
From income_table WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY result desc LIMIT 10;
"""
cur.execute(query)
cur.fetchall()

In [12]:
conn.commit()
query = """
Select symbol, cast(researchAndDevelopmentExpenses as float) / NULLIF(revenue, 0) as result
From income_table WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY result desc LIMIT 10;
"""
cur.execute(query)
cur.fetchall()

[('REGN', 0.3218745218957056),
 ('VRTX', 0.29481638040486435),
 ('MRK', 0.2824936450389632),
 ('BMY', 0.26207723787572323),
 ('ADSK', 0.259933420883853),
 ('NVDA', 0.2591133907309031),
 ('QCOM', 0.25392036037567467),
 ('LLY', 0.24799305617812697),
 ('TWTR', 0.23491093005527738),
 ('NOW', 0.22664689154779616)]

In [13]:
conn.commit()
query = """
"SELECT symbol, CAST(grossProfit AS float) / NULLIF(revenue, 0) AS ratio FROM income_table WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY ratio asc LIMIT 5;""SELECT date, CAST(researchAndDevelopmentExpenses AS float) / NULLIF(revenue, 0) AS ratio FROM income_table WHERE symbol = 'GOOGL' AND EXTRACT(YEAR FROM date) = '20 years'"
"""
cur.execute(query)
cur.fetchall()

[('AXP', 1.2637604415274464),
 ('SBUX', 0.9472744280976273),
 ('ANTM', 0.9420844034890495),
 ('HUM', 0.9353768388309248),
 ('BKNG', 0.914655679811654),
 ('UPS', 0.9092026279718296),
 ('TMUS', 0.9029782007982806),
 ('PYPL', 0.8466952549641092),
 ('ADSK', 0.7958647649879363),
 ('VZ', 0.7755277024288342)]

In [ ]:
conn.commit()
query = """
UPDATE income_table
SET researchAndDevelopmentExpensesPerRevenue=cast(researchAndDevelopmentExpenses as float) / NULLIF(revenue, 0);
"""
cur.execute(query)
conn.commit()

In [43]:
conn.commit()
query = """SELECT ceo FROM profile_table WHERE symbol = 'UBER'"""
cur.execute(query)
cur.fetchall()

[('Mr. Dara Khosrowshahi',)]

In [55]:
conn.commit()
query = """
DELETE FROM profile_table WHERE symbol='BRK-B';
"""
cur.execute(query)
conn.commit()

In [20]:
query = """
DELETE FROM profile_table;
"""
cur.execute(query)
query = """
DELETE FROM income_table;
"""
cur.execute(query)
query = """
DELETE FROM balance;
"""
cur.execute(query)
conn.commit()

In [30]:
# Commit the transaction.
conn.commit()

In [36]:
# To delete all rows, run """DELETE FROM income;"""
# To delete the table, run """DROP TABLE income;"""